In [1]:
import pandas as pd
import numpy as np
import zipfile
import requests
from io import BytesIO
import os
from sqlalchemy import create_engine
import json

In [2]:
# Cria o diretório
os.makedirs('./enem2019', exist_ok=True)

In [3]:
"""
# Define a url
url = 'http://download.inep.gov.br/microdados/microdados_enem_2019.zip'

# Download do conteudo
filebytes = BytesIO(
    requests.get(url, stream=True).content
)

# Extrarir o conteudo do zipfile
myzip = zipfile.ZipFile(filebytes)
myzip.extractall('./enem2019')
"""

"\n# Define a url\nurl = 'http://download.inep.gov.br/microdados/microdados_enem_2019.zip'\n\n# Download do conteudo\nfilebytes = BytesIO(\n    requests.get(url, stream=True).content\n)\n\n# Extrarir o conteudo do zipfile\nmyzip = zipfile.ZipFile(filebytes)\nmyzip.extractall('./enem2019')\n"

In [ ]:
# Liberar memoria
del(filebytes)

In [4]:
csv_file = './enem2019/DADOS/MICRODADOS_ENEM_2019.csv'

In [5]:
!pip install dask

In [9]:
import dask.dataframe as dd

In [10]:
enem = dd.read_csv('./enem2019/DADOS/MICRODADOS_ENEM_2019.csv', sep = ';', decimal=',', encoding='latin-1')

In [11]:
enem = enem.loc[enem.SG_UF_RESIDENCIA == 'MG']

In [12]:
enem.head(10)

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
1044,190001005671,2019,3170206,Uberlândia,31,MG,21,M,1,1,...,B,A,A,D,A,A,D,B,B,B
5234,190001009861,2019,3111705,Canaã,31,MG,20,M,1,1,...,A,A,A,B,A,A,C,A,A,A
16995,190001021622,2019,3104007,Araxá,31,MG,24,M,1,2,...,A,A,A,C,B,A,C,A,A,B
22008,190001026636,2019,3106200,Belo Horizonte,31,MG,33,M,1,1,...,B,A,B,B,A,A,D,A,D,B
22114,190001026742,2019,3162401,São João da Ponte,31,MG,23,M,1,3,...,A,A,A,B,B,A,C,A,B,A
41418,190001046046,2019,3124302,Espinosa,31,MG,22,M,1,3,...,B,A,A,C,B,A,E,A,B,B
41845,190001046473,2019,3169307,Três Corações,31,MG,19,F,1,3,...,B,A,A,B,B,B,E,A,B,B
48454,190001053082,2019,3106705,Betim,31,MG,20,M,1,3,...,B,A,A,B,A,A,D,A,B,B
51498,190001056126,2019,3143302,Montes Claros,31,MG,23,F,1,2,...,B,A,A,B,A,A,C,A,C,B
54094,190001058722,2019,3169307,Três Corações,31,MG,18,F,1,1,...,B,A,A,C,B,A,D,A,C,B


In [86]:
number_lines = sum(1 for row in (open(csv_file, encoding='latin-1')))

In [87]:
# FROM https://gist.github.com/jrivero/1085501

def split(filehandler, delimiter=',', row_limit=1000,
          output_name_template='output_%s.csv', output_path='.', keep_headers=True):
    import csv
    reader = csv.reader(filehandler, delimiter=delimiter)
    current_piece = 1
    current_out_path = os.path.join(
        output_path,
        output_name_template % current_piece
    )
    current_out_writer = csv.writer(open(current_out_path, 'w'), delimiter=delimiter)
    current_limit = row_limit
    if keep_headers:
        headers = next(reader)
        current_out_writer.writerow(headers)
    for i, row in enumerate(reader):
        if i + 1 > current_limit:
            current_piece += 1
            current_limit = row_limit * current_piece
            current_out_path = os.path.join(
                output_path,
                output_name_template % current_piece
            )
            current_out_writer = csv.writer(open(current_out_path, 'w'), delimiter=delimiter)
            if keep_headers:
                current_out_writer.writerow(headers)
        current_out_writer.writerow(row)

In [88]:
split(open(csv_file, encoding='latin-1'), delimiter=';', row_limit=1000000,
     output_name_template='microdados_enem_%s.csv', output_path='./enem2019/DADOS/splitted_files', keep_headers=True)

In [89]:
directory = './enem2019/DADOS/splitted_files/'

for filename in sorted(os.listdir(directory)):
    df = pd.read_csv(directory+filename, sep = ';', decimal = '.', encoding='latin-1')
    df = df.loc[
    df.SG_UF_RESIDENCIA == 'MG'
    ]
    os.remove(directory+filename)
    df.to_csv(directory+filename, index=False, sep = ';', decimal = '.', encoding='latin-1')

In [90]:
# Get access variables
with open('vaccess_pg.txt') as access_file:
    vaccess = json.load(access_file)

In [91]:
# Create connection
conn_str = 'postgresql+psycopg2://'+vaccess['user']+':'+vaccess['password']+'@'+vaccess['host:port']+'/'+vaccess['dbname']
engine_pg = create_engine(conn_str)

In [92]:
import psycopg2
conn = psycopg2.connect("host=172.18.0.2 dbname=postgres user=postgres password=postgres")
cur = conn.cursor()


directory = './enem2019/DADOS/splitted_files/'

try:
    for filename in sorted(os.listdir(directory)):
        print(filename)
        if filename.endswith(".csv"):
            enem = pd.read_csv(
                    directory+filename, sep = ';', decimal = '.', encoding='latin-1'
            )

            with open(directory+filename, 'r') as f:
                # Notice that we don't need the `csv` module.
                next(f) # Skip the header row.
                cur.copy_from(f, 'enem', sep=';', null="")

            conn.commit()

        else:
            continue
finally:
    cur.close()
    conn.close()

microdados_enem_1.csv
microdados_enem_2.csv
microdados_enem_3.csv
microdados_enem_4.csv
microdados_enem_5.csv
microdados_enem_6.csv
